In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import trino
from pprint import pprint
from collections import defaultdict 

In [2]:
def query_from_db(db, query, dataframe:bool=True):
    if db.lower() == 'presto':
        conn=trino.dbapi.connect(host='presto.bstis.com',port=8080,user='hadoop',catalog='hive', schema='prod')
    if db.lower() == 'mysql':
        conn=mysql.connector.connect(user='frontoffice', password='bis12345',
                                     host='frontoffice-aws-dev.cav6rhg74o38.us-west-2.rds.amazonaws.com',database='hyperloop')
    cur = conn.cursor()
    cur.execute(query)
    try:
        res = cur.fetchall()
        if dataframe==True:
            col_names = list(i[0] for i in cur.description)
            resultDf = pd.DataFrame(data=res, columns = col_names)
            return resultDf
        else:
            return res
    except Exception as e:
        return e

In [3]:
query_1 = "select distinct hl_site from hyperloop.job_log where hl_user != 'pentaho' and hl_table like 'asl_calcs,%' and hl_group='' and hl_status='Done' and hl_site !=''"

In [4]:
tenants = query_from_db('mysql', query_1, False)

In [5]:
tenantCheckList = [tenant[0] for tenant in tenants]

In [6]:
def get_paras(tenants, feed, result_list=[]):
    #print("len of result_list: %d"%len(result_list))
    for tenant_id, tenant in enumerate(tenants):   
        #print("tenant_id: %d"%tenant_id)
        if feed.lower() == 'elig':
            query = "select tenantid, tpa, params from {}.etl_log2 \
                     where status ='finished' and task_name='full_elig_incr' and params like 'process_type=elig%' order by process_time desc limit 1".format(tenant)
        elif feed.lower() == 'med':
            query = "select tenantid, tpa, params from {}.etl_log2 where status ='finished' and task_name !='post-process' \
                     and params like 'process_type=med%' or params like '%process_type=med%' order by process_time desc limit 1".format(tenant)
        elif feed.lower() == 'rx':
            query = "select tenantid,tpa, params from {}.etl_log2 where status ='finished' and task_name !='post-process' and params \
                     like 'process_type=rx%' or params like '%process_type=rx%' order by process_time desc limit 1".format(tenant)
        else:
            return ('Feed Not Found!')
        
        try:
            #changed this part so that the exception of not having the user is db is caught. will append the error into the result_list
            result = query_from_db('presto', query, False)
            
            if len(result)==0:
                result_list.append("Manual check: "+ tenant)
            else:
                result_list.append(result)
        except Exception as e:
            result_list.append(e)
    
    return result_list

In [7]:
elig_result = get_paras(tenantCheckList, 'elig', [])

In [8]:
med_result = get_paras(tenantCheckList, 'med', [])

In [9]:
rx_result = get_paras(tenantCheckList, 'rx', [])

In [10]:
def extract_info(pieces, info_dict=defaultdict(list), manual_check_list= []):
    error_check_list= []
    for p in pieces:
        try:
            if p[0] =='M':
                manual_check_list.append(p)
            else:
                tenantid = p[0][0]
                #print(p)
                tpaid = p[0][1]
                for i in p[0][2].split('|'):
                    if i.startswith('yaml_file='):
                        yaml = i
                        info_dict[yaml].append(tenantid)
        except Exception:
            #pass
            # edited this part so that this function returns a list of tenants that are not present in the db
            error_check_list.append(p)
            #print(p)
    return info_dict, manual_check_list, error_check_list

In [11]:
de, me, error1 = extract_info(elig_result,defaultdict(list), [])

In [12]:
dm, mm, error2 = extract_info(med_result,defaultdict(list), [])

In [13]:
dr, mr, error3 = extract_info(rx_result,defaultdict(list), [])

In [19]:
with pd.option_context('display.max_rows', None,) :
    display(pd.DataFrame(de.items(), columns = ['key', 'values']))

,key,values
0,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,[clevel]
1,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[compre, celebra, voya, sutherland, frcolle, b..."
2,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[cityofnoblesville, envoym]"
3,yaml_file=s3://com.bstis.tpafiles/input/hcginc...,[hcginc]
4,yaml_file=s3://com.bstis.tpafiles/input/basfco...,[basfco]
5,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,[nvgmines]
6,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[gulfwi, dexima]"
7,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[fidbui, monumen]"
8,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[ochsne, slidel, southern, terreb]"
9,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[danalim, indyps, kemkre, madisoncounty, raysk..."


In [20]:
with pd.option_context('display.max_rows', None,) :
    display(pd.DataFrame(dm.items(), columns = ['key', 'values']))
    

,key,values
0,yaml_file=bcbsnc,"[labcor, mwcomp]"
1,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,[clevel]
2,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[compre, celebra, voya, frcolle, otodevelopmen..."
3,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[cityofnoblesville, arling, centralsupply, gle..."
4,yaml_file=s3://com.bstis.tpafiles/input/hcginc...,[hcginc]
5,yaml_file=s3://com.bstis.tpafiles/input/basfco...,[basfco]
6,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[nvgmines, brownroot, metals, polyme]"
7,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,[sutherland]
8,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[gulfwi, dexima, orthoassmich]"
9,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[fidbui, monumen]"


In [21]:
with pd.option_context('display.max_rows', None,) :
    display(pd.DataFrame(dr.items(), columns = ['key', 'values']))

,key,values
0,yaml_file=optum7,[labcor]
1,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[southe, husqva]"
2,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[clevel, dexima, pearc, brownroot, guggenh, fl..."
3,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[compre, cityofnoblesville, arling, bucgrp, ca..."
4,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[hcginc, gulfwi, amtubg, armstro, broussa, bry..."
5,yaml_file=s3://com.bstis.tpafiles/input/basfco...,[basfco]
6,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[nvgmines, knaufin, macallister]"
7,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,"[celebra, frcolle, johnso, hfillc, medvet, sal..."
8,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,[voya]
9,yaml_file=s3://com.bstis.tpafiles/windfish/yam...,[sutherland]
